VGG16卷积网络详解
机器学习基础知识：
    1.相对熵(KL散度)：
        两个概率分布（probability distribution）间差异的非对称性度量
        衡量任意一个分布偏离真实分布的程度，如果两个分布完全匹配，那么KL散度为0，否则KL散度取值为0到无穷大之间
        https://blog.csdn.net/weixin_38026379/article/details/96114545


神经网络基础知识：
    1.卷积：
        卷积核：卷积操作中的一个过滤器，用于提取我们图像的特征
        卷积核大小：大小一般选择3x3和5x5，比较常用的是3x3
        卷积核参数：
            卷积核里面的每个值就是我们需要训练模型过程中的神经元参数（权重），开始会有随机的初始值，
            当训练网络时，网络会通过后向传播不断更新这些参数值，知道寻找到最佳的参数值。对于如何判断参数值的最佳,则是通过loss损失函数来评估.

    2.常见的loss函数  ：
        最小二乘法,
        交叉熵,
        极大似然估计

    3.卷积中的padding
        作用：保持尺度不变,或者是为了利用卷积不到的边界,卷积操作之后维度变少，得到的矩阵比原来矩阵小，这样不好计算，而我们只是希望作卷积，所以我们需要Padding
        边界数值的特征提取次数相对较少，为了能更好的把边界数值也利用上，所以给原始数据矩阵的四周都补上一层0
        计算卷积后保持尺度不变所需要的padding值
            假设原始图片为 H x W  卷积核大小为 x  ,步长为s
            则为了保持卷积后图像尺度不变

    4.池化pooling
        作用：池化操作相当于降维操作,降低数据量和计算量
        最大池化：一般都采用最大池化,个人猜想是最大池化保留了边缘信息,关键信息
        平均池化

    5.Flatten
        作用：将高维数据拉开,降维1维数据,方便输入全连接层

    6.Dropout
        作用：可以防止模型训练过拟合的情况
        原理：训练过程中，按照一定的比例将网络中的神经元进行丢弃,模拟人的记忆遗忘特性

VGG-16 (Very Deep Convolutional Networks for Large-scale Image Recongnition)
    VGG系列有很多文章,VGG-16特指层数为16层的那一篇
    16 = 13
    论文结论
        1.在一定范围内，通过增加深度能有效地提升网络性能；
        2.最佳模型：VGG16，从头到尾只有3x3卷积与2×2池化，简洁优美；
        3.多个小卷积核比单个大卷积核性能好（与alxnet对比可知）；
        4.AlexNet曾经用到的LRN层并没有带来性能的提升，因此在其它组的网络中均没再出现LRN层；
        5.尺度抖动scale jittering（多尺度训练，多尺度测试）有利于网络性能的提升。
    VGG-16发展历史：
        LeNet - 3个卷积层 + 2个下采样 + 1个FC
        AlexNet - 5个卷积层 + 3个FC + 1个softmax
        VGG-16 - 5个卷积层 + 5个maxpool + 3FC +1个softmax

    VGG16网络结构：
     注意：描述方式采用原文的描述方式, conv(接受大小范围) - (通道数量)
     原文：denoted as “conv <receptive field size> - <number of channels>"
     input 为244x244的RGB图，所以是三个维度 244x244x3


    【input 244x244】 ->【conv（3）-（64） ，con（3）-（ 64）】->【maxpool】 3层 特征维度3->64
     ->【conv（3）-（128） ，con（3）-（128）】->【maxpool】 2层 特征维度64->128
     ->【conv（3）-（256） ，con（3）-（256），conv（3）-（256）】->【maxpool】 2层 特征维度128->256
     ->【conv（3）-（512） ，con（3）-（512），conv（3）-（512）】->【maxpool】 2层 特征维度256->512
     ->【conv（3）-（512） ，con（3）-（512），conv（3）-（512）】->【maxpool】 2层 特征维度512->512
     ->  【FC-(4096)】  ->   【FC-(4096)】  ->  【FC-(1000)】 -> 【softmax】 ->【output】 5层  特征维度 4096->1000->概率【0，1】

     总共 3+2+2+2+2+5 = 16层

In [6]:
a = 3 // 2
a

1

In [3]:
class SRCNN (nn.Module):  # SRCNN为继承 nn.moudule的子类
    def __init__(self, num_channels=1):
        super(SRCNN, self).__init__()   #重新定义SRCNN
        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=(9,9), padding='same')  # 卷积之前加在输入的两边的padding。输入的两边添加的填充。默认值：0
        self.conv2 = nn.Conv2d(64, 32, kernel_size=(1,1))
        self.conv3 = nn.Conv2d(32, num_channels, kernel_size=(5,5), padding= int(5/2))
        #self.relu = nn.ReLU(inplace=True)  # 整流 max（0，fx）
        self.relu = nn.PReLU() # 龚泊诚是pig

        '''
            conv1 : 64 个 9x9
            conv2 ：32 个 1x1
            conv3 ：1个 5x5
        '''
        '''PReLU是LReLU(leak relu)的改进，可以自适应地从数据中学习参数。
            PReLU具有收敛速度快、错误率低的特点。PReLU可以用于反向传播的训练，可以与其他层同时优化。'''

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.conv3(x)
        return x

3

In [ ]:
'''依据原始论文搭建VGG网络'''
import torch.nn as nn
import torch
class VGG_16(nn.Module):
     def __init__(self, num_channels=3):  #由于传入的是RGB 所以传入通道为3
       super(VGG_16, self).__init__()
       #【input 244x244】 ->【conv（3）-（64） ，con（3）-（ 64）】->【maxpool】 3层 特征维度3->64
       self.conv1 = nn.Conv2d(in_channels=num_channels, out_channels=64, kernel_size=(3,3), padding='same')
       self.conv2 = nn.Conv2d(64, 64, (3,3), padding='same')
       #->【conv（3）-（128） ，con（3）-（128）】->【maxpool】 2层 特征维度64->128
       self.conv3 = nn.Conv2d(64, 128, (3,3), padding='same')
       self.conv4 = nn.Conv2d(128, 128, (3,3), padding='same')
       # ->【conv（3）-（256） ，con（3）-（256），conv（3）-（256）】->【maxpool】 2层 特征维度128->256
       self.conv5 = nn.Conv2d(128,256,(3,3),padding='same')
       self.conv6 = nn.Conv2d(256,256,(3,3),padding='same')
       self.conv7 = nn.Conv2d(256,256,(3,3),padding='same')
       #->【conv（3）-（512） ，con（3）-（512），conv（3）-（512）】->【maxpool】 2层 特征维度256->512
       self.conv8 = nn.Conv2d(256,512,(3,3),padding='same')
       self.conv9 = nn.Conv2d(512,512,(3,3),padding='same')
       self.conv10 = nn.Conv2d(512,512,(3,3),padding='same')
       #->【conv（3）-（512） ，con（3）-（512），conv（3）-（512）】->【maxpool】 2层 特征维度512->512
       self.conv11 = nn.Conv2d(512,512,(3,3),padding='same')
       self.conv12 = nn.Conv2d(512,512,(3,3),padding='same')
       self.conv13 = nn.Conv2d(512,512,(3,3),padding='same')
       self.relu = nn.ReLU()
       self.maxpool = nn.MaxPool2d(kernel_size=(2,2),stride=2)
       self.linear = nn.Linear()


       def forward(self,input):  # input 为 RGB 图片，此时我们关注一下图片的大小
           temp  = self.relu(self.conv1(input)) #244x244
           temp  = self.relu(self.conv2(temp))
           temp  = self.maxpool(temp) #
           temp  = self.relu(self.conv3(temp))
           temp  = self.relu(self.conv4(temp))
           temp  = self.maxpool(temp)
           temp  = self.relu(self.conv5(temp))
           temp  = self.relu(self.conv6(temp))
           temp  = self.relu(self.conv7(temp))
           temp  = self.maxpool(temp)
           temp  = self.relu(self.conv8(temp))
           temp  = self.relu(self.conv9(temp))
           temp  = self.relu(self.conv10(temp))
           temp  = self.maxpool(temp)
           temp  = self.relu(self.conv11(temp))
           temp  = self.relu(self.conv12(temp))
           temp  = self.relu(self.conv13(temp))
           temp  = self.maxpool(temp)